In [28]:
import sqlite3
import csv

'''
全局配置
'''
db_file = r"E:\gis\data\440100-trip_resident\Mobile Signaling.db"
table_name = 'data'


In [29]:
'''
测试
'''

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute("SELECT * FROM data LIMIT 20")
rows = cursor.fetchall()

column_names = [description[0] for description in cursor.description]
print(", ".join(column_names))
for row in rows:
    print(row)

conn.close()


record_date, travel_period, purpose, city, o_lng, o_lat, d_lng, d_lat, user_quantity, hour
(20241020, '16:00-16:30', '其家', 440100, 112.9875, 23.4475, 113.1975, 23.1575, 2, 16)
(20241020, '22:00-22:30', '其家', 440100, 112.9875, 23.4475, 113.1975, 23.1575, 1, 22)
(20241020, '17:30-18:00', '其其', 440100, 112.9875, 23.4475, 113.2125, 23.1175, 1, 17)
(20241020, '17:30-18:00', '其其', 440100, 112.9875, 23.4475, 113.2325, 23.2625, 1, 17)
(20241020, '16:00-16:30', '其家', 440100, 112.9875, 23.4475, 113.2375, 23.1925, 1, 16)
(20241020, '19:00-19:30', '其其', 440100, 112.9875, 23.4475, 113.2425, 23.1475, 1, 19)
(20241020, '19:00-19:30', '其其', 440100, 112.9875, 23.4475, 113.2425, 23.1475, 1, 19)
(20241020, '14:30-15:00', '其家', 440100, 112.9875, 23.4475, 113.2425, 23.1525, 1, 14)
(20241020, '17:30-18:00', '其其', 440100, 112.9875, 23.4475, 113.2425, 23.1525, 1, 17)
(20241020, '19:30-20:00', '其家', 440100, 112.9875, 23.4475, 113.2425, 23.1525, 1, 19)
(20241020, '20:00-20:30', '其其', 440100, 112.9875, 23.4475, 

In [31]:
'''
查看行数
'''

with sqlite3.connect(db_file) as conn:
    cursor = conn.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_num = cursor.fetchone()[0]
print(f"表 '{table_name}' 的行数: {row_num:,}")


表 'data' 的行数: 3,635,466


In [ ]:
'''
手机信令处理: 合并相同坐标和user_quantity,删除o坐标和travel_period
'''

cursor.execute('''
            CREATE TEMP TABLE temp_travel_data AS
            SELECT 
                record_date,
                d_lng,
                d_lat,
                SUM(user_quantity) AS user_quantity
            FROM 
                data
            GROUP BY 
                record_date, d_lng, d_lat;
        ''')
cursor.execute('DELETE FROM data;')
cursor.execute('''
    INSERT INTO data (record_date, d_lng, d_lat, user_quantity)
    SELECT record_date, d_lng, d_lat, user_quantity FROM temp_travel_data;
''')
cursor.execute('DROP TABLE temp_travel_data;')

conn.commit()
print("数据处理完成并已提交。")
if conn:
    conn.close()
    print("数据库连接已关闭。")


In [ ]:
'''
1,将user_quantity的值从大到小绘制
2,循环头尾分割法
'''

import matplotlib.pyplot as plt
import numpy as np


def plot_above_average_data(db_file):
    cursor.execute("SELECT user_quantity FROM data")
    user_quantity_list = [row[0] for row in cursor.fetchall()]
    sorted_data = sorted(user_quantity_list, reverse=True)
    # 计算平均值并过滤小于平均值的数据
    while True:
        if not sorted_data:
            print("数据为空，无法继续处理")
            return None
        ave = sum(sorted_data) / len(sorted_data)
        print(f"当前平均值: {ave}")
        # 找到第一个小于平均值的索引
        clipnum = next((i for i, j in enumerate(sorted_data) if j < ave), len(sorted_data))
        filtered_data = sorted_data[:clipnum]
        print(f"过滤前数据点数: {len(sorted_data)}")
        print(f"过滤后数据点数: {len(filtered_data)}")
        # 绘制折线图
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(filtered_data) + 1), filtered_data, 'o-', color='b')
        plt.title("休憩人数降序排列")
        plt.xlabel("排名")
        plt.ylabel("休憩人数")
        plt.grid(True, linestyle='--', alpha=0.7)
        # 动态设置y轴刻度，避免过密
        max_val = max(filtered_data) if filtered_data else 0
        plt.yticks(np.arange(0, max_val + 1, max(1000, max_val // 10)))
        plt.tight_layout()
        plt.show()
        
        user_input = input("是否继续过滤数据?输入True继续,其他任意键退出: ").strip().lower()
        if user_input != 'true':
            break
        
        sorted_data = filtered_data

    conn.close()
    return filtered_data


p1 = r"E:\gis\data\440100-trip_resident\destination.db"
# 第一次绘制
conn = sqlite3.connect(p1)
cursor = conn.cursor()
cursor.execute("SELECT user_quantity FROM data")
user_quantity_list = [row[0] for row in cursor.fetchall()]
sorted_data = sorted(user_quantity_list, reverse=True)

plt.rcParams["font.family"] = ["SimHei"]
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(sorted_data) + 1), sorted_data, 'o-', color='b')
plt.title("休憩人数降序排列")
plt.xlabel("排名")
plt.ylabel("休憩人数")
plt.grid(True, linestyle='--', alpha=0.7)
plt.yticks(np.arange(0, 35000, 1000))
plt.xticks(np.arange(0, 6000, 500)) 
plt.tight_layout()
plt.show()
plot_above_average_data(db_file=p1)


In [ ]:
'''
sqlite导出为csv
'''

export_csvpath = r'data\export.csv'
with open(export_csvpath, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    with sqlite3.connect(db_file) as conn:
        cursor = conn.execute(f"SELECT * FROM {table_name}")
        batch_data = cursor.fetchmany(10000)
        writer.writerow([i[0] for i in cursor.description])
        count = 0
        while batch_data:
            writer.writerows(batch_data)
            batch_data = cursor.fetchmany(10000)
            count += len(batch_data)
        print(f'写入了{count}条数据')
        print(f'写入完成')


写入了3625466条数据
写入完成
